# Analysis of GR immunofluorescence data (cell sorting)
This Jupyter notebook contains scripts used to analyze immunofluorescence data. Specifically, the data is the median anti-GR nuclear and cytoplasmic signals from each imaged cell. The nuclear signal is then plotted for the 30 minute dexamethasone treated and ethanol treated samples.

# Table of contents
1. [Load packages and files and format data](#load-packages-and-files-and-format-data)
2. [Plot box plots of nuclear GR signal](#plot-box-plots-of-nuclear-gr-signal)

## Load packages and files and format data
Load required packages and results files. Additionally, format data into a single dataframe for plotting.

In [ ]:
#notebook for plotting IF data
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
#file paths here
if_path="..\\data"
samples = ["Parental","wt GR", "SoF", "Ctrl"]
times = ["EtOH","30 min Dex"]
if_files_untreated = ["U2OS_UT_2.xlsx",
						"U2OS_UT_4.xlsx",
						"U2OS_UT_8.xlsx",
						"U2OS_UT_11.xlsx"]
if_files_treated = ["U2OS_T_2.xlsx",
						"U2OS_T_4.xlsx",
						"U2OS_T_8.xlsx",
						"U2OS_T_11.xlsx"]
if_files = [[if_path+file for file in if_files_untreated],
			[if_path+file for file in if_files_treated]]
#take data and put in dataframes
def formatData(file,sample,time):
	data=pd.read_excel(file)
	nuc=np.array(data["gfp_nuc_median"]) 
	cyto=np.array(data["gfp_cyto_median"])
	nucfrac=[x/(x+y) for x,y in zip(nuc,cyto)]
	dict={"Nuclear signal":nuc,
		"Cytoplasmic signal":cyto,
		"Nuclear fraction":nucfrac,
		"Cell line":sample,
		"Treatment":time
		}
	df=pd.DataFrame(dict)
	return df
#format data from each data file and concatenate into one dataframe
dfs=[]
for i,time in enumerate(times):
	for n,sample in enumerate(samples):
		file=if_files[i][n]
		dfs.append(formatData(file,sample,time))
plotdf=pd.concat(dfs)

## Plot box plots of nuclear GR signal

In [ ]:
#box plots for nuclear GR signal
def boxNuc(data):
	fontsize=8
	mm=0.0393701
	figwidth=80
	figheight=50
	plt.rcParams["font.family"]="Arial"
	plt.rcParams["font.size"]=fontsize
	fig=plt.figure(dpi=300,figsize=(figwidth*mm,figheight*mm))
	palette=["#D647E6","#6389FF","#FF5C5C","#1AC11B"]
	#fliers are not plotted due to large variance in signal
	g=sns.boxplot(data=data, x="Treatment", y="Nuclear signal", hue="Cell line",palette=palette,
				linewidth=figwidth/80,fliersize=0)
	plt.xlabel("Treatment")
	plt.ylabel("Nuclear signal")
	plt.ylim(0,10000)
	plt.legend(loc="upper left", fontsize=fontsize-1)
	plt.show()
boxNuc(plotdf)
